In [12]:
import github_fetching as fetcher
import data_cleaning as cleaner
import sentiment_analysis as analyzer
import visualization as visualizer
import pandas as pd
import matplotlib.pyplot as plt
import os
from collections import defaultdict
import shutil
import requests
import numpy as np

In [ ]:
hugging_face_folder = "inter_rater/hugging_face/"

In [ ]:
fetcher.fetch_issues_pr(repo_name = 'tukaani-project/xz', folder_location = os.path.join(hugging_face_folder, "individual_issue_PR/"), start_year = 2023, start_month = 1, start_day = 1, end_year = 2023, end_month = 12, end_day = 31)
cleaner.clean_thread(folder_path = os.path.join(hugging_face_folder, "individual_issue_PR/"))

In [ ]:
user_interactions = analyzer.sentence_sentiment_analysis(source_path = hugging_face_folder)


**IBM WATSON**

In [ ]:
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions, EmotionOptions
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
# uoft
apikey_uoft="AmELNIND2iUq599YVQrEO6fhvGTThbnrWUk-5CkMNhwM"
url_uoft = "https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/e4672fef-a075-4109-9250-f22368dce7cb"

# gmail
apikey = 'dfaTQ4Cc3XVSEM41zmH4jOG0ZGxY8j_fp0Y2t01LoHtR'
url = 'https://api.us-east.natural-language-understanding.watson.cloud.ibm.com/instances/51d76a3e-11e2-4540-b818-55bdedb8e7fa'

# Set up the authenticator
authenticator = IAMAuthenticator(apikey_uoft)

# Set up the Natural Language Understanding instance
nlu = NaturalLanguageUnderstandingV1(
    version='2021-08-01',
    authenticator=authenticator
)
nlu.set_service_url(url_uoft)

In [ ]:
def group_text_from_csv(directory_path, column_name='body'):
    grouped_text = ""
    column_name = 'body'

    df = pd.read_csv(file_path)
    grouped_text += "".join(df[column_name].dropna()) + "\n"

    return grouped_text

def analyze_sentiment_emotion(text, nlu_instance):
    try:
        # Check if the text is non-empty and has sufficient length
        if len(text.strip()) < 10:  # Adjust the length threshold as needed
            return 'N/A', 0.0, {}

        response = nlu_instance.analyze(
            text=text,
            features=Features(
                sentiment=SentimentOptions(),
                emotion=EmotionOptions()
            ),
            language='en'
        ).get_result()

        sentiment = response['sentiment']['document']['label']
        sentiment_score = response['sentiment']['document']['score']
        emotions = response['emotion']['document']['emotion']

        return sentiment, sentiment_score, emotions
    
    except Exception as e:
        print(f"Error processing text: {str(e)}")
        return 'Error', 0.0, {}

def get_dominant_emotion(emotion_dict):
    if isinstance(emotion_dict, dict):
        # Find the emotion with the highest score
        dominant_emotion = max(emotion_dict, key=emotion_dict.get)
        return dominant_emotion
    else:
        return 'N/A'  # In case there's an issue with the emotion data

def classify_pr_or_issue(filename):
    if 'pr' in filename.lower():
        return 'PR'
    elif 'issue' in filename.lower():
        return 'Issue'
    else:
        return 'Unknown'

def is_valid_author(author_name):
    # Ensure the input is a string; if not, return False
    if not isinstance(author_name, str):
        return False
    
    # Check if the name contains only letters, spaces, hyphens, apostrophes, or digits
    return bool(re.match(r'^[a-zA-Z0-9\s\'-]+$', author_name))

def count_messages(column):
    def safe_eval(x):
        try:
            return len(ast.literal_eval(x))
        except (ValueError, SyntaxError):
            return 0  # Return 0 if there's an error in parsing
    return column.apply(safe_eval)

def count_dataset(user_interactions):
    num_pairs = len(user_interactions)
    num_messages = 0
    
    for index, row in user_interactions.iterrows():
        num_messages = num_messages + len(row['positive']) + len(row['negative']) + len(row['neutral'])
        
    print("Number of author pairs: " + str(num_pairs))
    print("Number of messages: " + str(num_messages))

def get_github_user_name(username, token = os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN")):
    url = f"https://api.github.com/users/{username}"
    headers = {"Authorization": f"token {token}"}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('name') == None:
            return username
            
        return data.get('name', username)  # Return username if the real name is not found
    else:
        print(f"Error: Unable to fetch data for username '{username}'. Status code: {response.status_code}")
        return username

In [ ]:
sentiment_tracker = defaultdict(lambda: {'positive': [], 'neutral': [], 'negative': []})

def update_sentiment_tracker(source_author, target_in_message, sentiment, sentiment_score, original_message, filename):
    if sentiment == 'positive':
        sentiment_tracker[(source_author, target_in_message)]['positive'].append({
            'message': original_message,
            'score': sentiment_score,
            'file_name' : filename
        })
    elif sentiment == 'neutral':
        sentiment_tracker[(source_author, target_in_message)]['neutral'].append({
            'message': original_message,
            'score': sentiment_score,
            'file_name' : filename
        })
    elif sentiment == 'negative':
        sentiment_tracker[(source_author, target_in_message)]['negative'].append({
            'message': original_message,
            'score': sentiment_score,
            'file_name' : filename
        })

In [ ]:
ibm_folder = "inter_rater/ibm/"

In [ ]:
fetcher.fetch_issues_pr(repo_name = 'tukaani-project/xz', folder_location = ibm_folder, start_year = 2023, start_month = 1, start_day = 1, end_year = 2023, end_month = 12, end_day = 31)
cleaner.clean_thread(folder_path = os.path.join(ibm_folder, "individual_issue_PR/"))

In [ ]:
issue_pr_path = os.path.join(ibm_folder, "individual_issue_PR/")
post_sentiment_results = []

for filename in os.listdir(issue_pr_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(issue_pr_path, filename)
        df = pd.read_csv(file_path)

        for index, row in df.iterrows():
            source_author = row['from']
            target_in_message = row['to']
            
            # Analyze the sentiment of the message body
            sentiment, sentiment_score, emotion = analyze_sentiment_emotion(row['body'], nlu)

            # Update the sentiment count in the tracker with the original message
            update_sentiment_tracker(source_author, target_in_message, sentiment, sentiment_score, row['body'],filename)

        print(filename + " is analyzed")

In [ ]:
result_list = []
for (source_author, target_in_message), sentiments in sentiment_tracker.items():
    result_list.append({
        'from': source_author,
        'to': target_in_message,
        'positive': sentiments['positive'],
        'neutral': sentiments['neutral'],
        'negative': sentiments['negative']
    })

user_interactions = pd.DataFrame(result_list)

directory = os.path.join(ibm_folder, "deleted_user_interactions/")
if not os.path.exists(directory):
    os.mkdir(directory)
else:
    shutil.rmtree(directory)
    os.mkdir(directory)

# Step 1: Remove invalid 'to' values
invalid_to = user_interactions[user_interactions['to'].isna()]
user_interactions = user_interactions.dropna(subset=['to'])
invalid_to.to_csv(os.path.join(directory, "removed_invalid_to.csv"), index=False)

# Step 2: Remove self-interactions
self_interactions = user_interactions[user_interactions['from'] == user_interactions['to']]
user_interactions = user_interactions[user_interactions['from'] != user_interactions['to']]
self_interactions.to_csv(os.path.join(directory, "removed_self_interactions.csv"), index=False)

# Step 3: Filter out rows where authors are below the contribution threshold
contribution_df = cleaner.calculate_author_contributions()
contribution_df = contribution_df.sort_values(by='count', ascending=False)
threshold = contribution_df['count'].describe()['50%']
authors_to_eliminate = contribution_df[contribution_df['count'] < threshold]['from']

below_threshold_interactions = user_interactions[
    user_interactions['from'].isin(authors_to_eliminate) |
    user_interactions['to'].isin(authors_to_eliminate)
]
user_interactions = user_interactions[
    ~user_interactions['from'].isin(authors_to_eliminate) &
    ~user_interactions['to'].isin(authors_to_eliminate)
]

below_threshold_interactions.to_csv(os.path.join(directory, "removed_below_threshold_interactions.csv"), index=False)

print("Done cleaning for user interactions")

user_interactions = user_interactions.sort_values(by=['from'])
user_interactions['from'] = user_interactions['from'].apply(get_github_user_name)
user_interactions['to'] = user_interactions['to'].apply(get_github_user_name)
print("Done GitHub username fetching")

In [ ]:
temp = analyzer.construct_individual_conversations(ibm_folder, user_interactions)

In [2]:
ibm_folder = "inter_rater/ibm/individual_conversations"
hugging_face_folder = "inter_rater/hugging_face/individual_conversations"

In [ ]:
 # Get list of files in both folders
ibm_files = sorted(os.listdir(ibm_folder))
hf_files = sorted(os.listdir(hugging_face_folder))

all_conversations = []

# Iterate over the files in both folders
for file1, file2 in zip(ibm_files, hf_files):
    # Ensure the files correspond (by name)
    if file1 != file2:
        raise ValueError(f"File mismatch: {file1} vs {file2}")

    # Load the files
    df1 = pd.read_csv(os.path.join(ibm_folder, file1))
    df2 = pd.read_csv(os.path.join(hugging_face_folder, file2))

    mood1 = df1['mood']
    mood2 = df2['mood']
    

    # Ensure moods are of the same length before comparing
    if len(mood1) != len(mood2):
        continue

    # Track sentiments and mismatches
    for i in range(len(mood1)):
            all_conversations.append({
                "from": df1['from'][i],
                "to": df1['to'][i],
                "file_name": df1['file_name'][i],
                "message": df1['message'][i],
                "rater1_mood": df1['mood'][i],
                "rater1_score": df1['score'][i],
                "rater2_mood": df2['mood'][i],
                "rater2_score":df2['score'][i]
            })

all_conversations = pd.DataFrame(all_conversations);
print(len(all_conversations))
all_conversations.to_csv("inter_rater/hf_ibm_combined.csv", index = False)



In [3]:
from sklearn.metrics import cohen_kappa_score
from statsmodels.stats.inter_rater import fleiss_kappa

In [8]:
irr_file = pd.read_csv("inter_rater/hf_ibm_chatgpt.csv")

In [15]:
# rater1: IBM, rater2: Hugging Face, rater3: Myself
ibm_hf_kappa_score = cohen_kappa_score(irr_file['rater1_mood'], irr_file['rater2_mood'])
ibm_human_kappa_score = cohen_kappa_score(irr_file['rater1_mood'], irr_file['rater3_mood'])
hf_human_kappa_score = cohen_kappa_score(irr_file['rater2_mood'], irr_file['rater3_mood'])

# Create a matrix where each row represents a message, and columns represent counts of each sentiment
ratings = []
sentiment_options = ['positive', 'neutral', 'negative']

for _, row in irr_file.iterrows():
    # Count the occurrences of each sentiment for each message
    sentiments = [row['rater1_mood'], row['rater2_mood'], row['rater3_mood']]
    counts = [sentiments.count(sent) for sent in sentiment_options]
    ratings.append(counts)

# Convert ratings to a numpy array
ratings_array = np.array(ratings)

# Calculate Fleiss' Kappa
fleiss_kappa_score = fleiss_kappa(ratings_array, method='fleiss')
print("Fleiss' Kappa:", fleiss_kappa_score)

print("\nIBM v.s. Hugging Face: " + str(ibm_hf_kappa_score))
print("\nIBM v.s. Human: " + str(ibm_human_kappa_score))
print("\nHugging Face v.s. Human: " + str(hf_human_kappa_score))

Fleiss' Kappa: 0.5140614840795245

IBM v.s. Hugging Face: 0.38161164359247435

IBM v.s. Human: 0.4892921960072595

Hugging Face v.s. Human: 0.7090209703437151
